In [4]:
import copy
import json
import math
import os
import pathlib
import time
import psycopg2
import requests
from requests.adapters import HTTPAdapter
from requests.auth import HTTPBasicAuth
from urllib3.util.retry import Retry

In [5]:
PL_API_KEY = 'PLAK721cf7576b5f4835bcd0f2dfe9a7a395'
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

SESSION = requests.Session()

SESSION.auth = (PL_API_KEY, '')
AUTH = HTTPBasicAuth(PL_API_KEY, '')

retries = Retry(total=10, backoff_factor=1, status_forcelist=[429])
SESSION.mount('https://', HTTPAdapter(max_retries=retries))

In [6]:
class PGConn:
    def __init__(self, host, port, dbname, user=None, passwd=None):
        self.host = host
        self.port = port
        self.dbname = dbname
        if user is not None:
            self.user = user
        else:
            self.user = ""
        if passwd is not None:
            self.passwd = passwd
        else:
            self.passwd = ""
        self.conn = None

    def connection(self):
        """Return connection to PostgreSQL.  It does not need to be closed
        explicitly.  See the destructor definition below.

        """
        if self.conn is None:
            conn = psycopg2.connect(dbname=self.dbname,
                                    host=self.host,
                                    port=str(self.port),
                                    user=self.user,
                                    password=self.passwd)
            self.conn = conn
            
        return self.conn

    def __del__(self):
        """No need to explicitly close the connection.  It will be closed when
        the PGConn object is garbage collected by Python runtime.

        """
        print(self.conn)
        self.conn.close()
        self.conn = None

pgconn_obj = PGConn(
    "localhost",
    5432,
    "dolr",
    "sameer",
    "swimgood"
)
    
pgconn=pgconn_obj.connection()

sql_query = f"""
    select
        st_astext(st_transform(geom, 4674)) as geom_text
    from
        pilot.dagdagad_farmplots_dedup
    where
        gid = 1
    ;
"""

with pgconn.cursor() as curs:
    curs.execute(sql_query)
    poly_fetch = curs.fetchone()[0]

poly_split = poly_fetch.split('(')
poly_coords = poly_split[3]
poly_coords = poly_coords.split(')')[0]
poly_point_coords = poly_coords.split(',')

points = []
for pt_coords in poly_point_coords:
    pt_split = pt_coords.split(' ')
    lat = float(pt_split[0])
    long = float(pt_split[1])
    points.append([lat, long])
print(points)

pgconn_obj.__del__()

UndefinedTable: relation "dagdagad.farmplots_dedup" does not exist
LINE 5:         dagdagad.farmplots_dedup
                ^


In [ ]:
def place_monthly_order(mosaic_name, points):
    order_params = {
        "name": "Basemap order with geometry",
        "source_type": "basemaps",
        "products": [
            {
                "mosaic_name": mosaic_name,
                "geometry":{
                "type": "Polygon",
                "coordinates":[
                    points
                ]
                }
            }
        ]
    }
    
    paramRes = requests.post(ORDERS_API_URL,
        data=json.dumps(order_params),
        auth=AUTH,
        headers={'content-type': 'application/json'}
    )

    print(paramRes.text)

    order_id = paramRes.json()['id']
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

def poll_for_success(order_url, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=AUTH)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)

def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))


In [ ]:
order_urls = []
for month in range(1, 13):
    if month < 10:
        month = "0" + str(month)
    else:
        month = str(month)
    mosaic_name = f"global_monthly_2022_{month}_mosaic"
    order_url = place_monthly_order(mosaic_name, points)
    order_urls.append(order_url)
    

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/0284429d-3a55-4e73-9e98-020164e0f6f1"},"created_on":"2024-01-14T16:59:48.525Z","error_hints":[],"id":"0284429d-3a55-4e73-9e98-020164e0f6f1","last_message":"Preparing order","last_modified":"2024-01-14T16:59:48.525Z","name":"Basemap order with geometry","products":[{"geometry":{"coordinates":[[[77.6226544,21.0217687],[77.6226003,21.0208757],[77.6221296,21.020901],[77.6222195,21.0221907],[77.622699,21.022120099999995],[77.6226544,21.0217687]]],"type":"Polygon"},"mosaic_name":"global_monthly_2023_01_mosaic"}],"source_type":"basemaps","state":"queued"}

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/603761e4-c5a9-446a-9a87-de62f202d4d9"},"created_on":"2024-01-14T16:59:49.389Z","error_hints":[],"id":"603761e4-c5a9-446a-9a87-de62f202d4d9","last_message":"Preparing order","last_modified":"2024-01-14T16:59:49.389Z","name":"Basemap order with geometry","products":[{"geometry":{"coordinates":[[[77.6226544,21.02176

In [ ]:
for order_url in order_urls:
    poll_for_success(order_url)
    r = requests.get(order_url, auth=AUTH)
    response = r.json()
    results = response['_links']['results']
    print([r['name'] for r in results])
    download_results(results)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
['0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_quad.tif', '0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_provenance_vector.zip', '0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_provenance_raster.tif', '0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_ortho_udm2.tif', '0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_metadata.json']
5 items to download
downloading 0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2023_01_mosaic/1465-1146_quad.tif to data\0284429d-3a55-4e73-9e98-020164e0f6f1\global_monthly_2023_01_mosaic\1465-1146_quad.tif
downloading 0284429d-3a55-4e73-9e98-020164e0f6f1/global_monthly_2